# Convolutional Neural Network (CNN)

We will focus on Convolutional Neural Network
The main goal of this laboratory is to solve a multiclass classification problem with 10 different classes


In [2]:
# import some libraries
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

In [3]:
# load the dataset
num_classes = 10
new_im_size = 32
channels = 3
cifar10 = tf.keras.datasets.cifar10
(x_learn, y_learn),(x_test, y_test) = cifar10.load_data()


In [4]:
# Normalize the data in [0 1]
print("Normalizing training set..")
x_learn = np.asarray(x_learn, dtype=np.float32) / 255 # Normalizing training set
print("Normalizing test set..")
x_test = np.asarray(x_test, dtype=np.float32) / 255 # Normalizing test set
# split in training and validation
x_train, x_val, y_train, y_val = train_test_split(x_learn, y_learn, test_size=0.25, random_state=12)

Normalizing training set..
Normalizing test set..


In [5]:
# Standardizing the data
# This process is useful to make the model converge faster

def standardize_dataset(X):
    image_means = []
    image_stds = []

    for image in X:
        image_means.append(np.mean(image)) # Computing the image mean
        image_stds.append(np.std(image)) # Computing the image standard deviation

    dataset_mean = np.mean(image_means) # Computing the dataset mean
    dataset_std = np.mean(image_stds) # Computing the dataset standard deviation
    return [dataset_mean, dataset_std] # For every image we subtract to it the dataset mean and we divide by the dataset standard deviation

dataset_mean, dataset_std = standardize_dataset(x_train)

print("Standardizing training set..")
x_train = (x_train-dataset_mean)/dataset_std # Standardizing the training set
print("Standardizing validation set..")
x_val = (x_val-dataset_mean)/dataset_std # Standardizing the test set
print("Standardizing test set..")
x_test = (x_test-dataset_mean)/dataset_std # Standardizing the test set

# one hot encode target values
y_train_enc = tf.keras.utils.to_categorical(y_train)
y_val_enc = tf.keras.utils.to_categorical(y_val)
y_test_enc = tf.keras.utils.to_categorical(y_test)

print("Size of the training set")
print("x_train", x_train.shape)
print("y_train", y_train.shape)

print("Size of the validation set")
print("x_val", x_val.shape)
print("y_val", y_val.shape)

print("Size of the test set")
print("x_test", x_test.shape)
print("y_test", y_test.shape)

Standardizing training set..
Standardizing validation set..
Standardizing test set..
Size of the training set
x_train (37500, 32, 32, 3)
y_train (37500, 1)
Size of the validation set
x_val (12500, 32, 32, 3)
y_val (12500, 1)
Size of the test set
x_test (10000, 32, 32, 3)
y_test (10000, 1)


### Training a model from scratch

In [6]:
# Creating the model from scratch
import tensorflow.keras
from tensorflow.keras import Sequential,Input,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from sklearn.metrics import accuracy_score

In [8]:
scratch_model = Sequential()

# Build here your keras model.
# Try to use one convolutional layer, joint with pooling layer and dropout layer

# Creating conv 1: conv with 1024 kernels of size 3x3, padding='same', input_shape=(new_im_size, new_im_size, channels)
# + LeakyReLU(alpha=0.1) + maxpooling with region size 2x2 and padding='same'+Dropout(0.25)
scratch_model.add(Conv2D(1024, kernel_size=(3, 3), padding='same',input_shape=(new_im_size, new_im_size, channels)))
scratch_model.add(LeakyReLU(alpha=0.1))
scratch_model.add(MaxPooling2D((2, 2),padding='same'))
scratch_model.add(Dropout(0.25))

scratch_model.add(Conv2D(254, kernel_size=(3, 3), padding='same',input_shape=(new_im_size, new_im_size, channels)))
scratch_model.add(LeakyReLU(alpha=0.1))
scratch_model.add(MaxPooling2D((2, 2),padding='same'))
scratch_model.add(Dropout(0.25))


# Adding the dense final part: Flatten + Dense with 64 neurons and relu + Dropout 25% + Dense with 10 neurons and softmax
scratch_model.add(Flatten())
scratch_model.add(Dense(64, activation='relu'))
scratch_model.add(Dropout(0.25))
scratch_model.add(Dense(num_classes, activation='softmax'))

# Compile the model with the Adam optimizer
scratch_model.compile(loss=tensorflow.keras.losses.categorical_crossentropy, optimizer=tensorflow.keras.optimizers.Adam(),metrics=['accuracy'])

# Visualize the model through the summary function
scratch_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 32, 32, 1024)      28672     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 32, 32, 1024)      0         
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 1024)      0         
 g2D)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 16, 16, 1024)      0         
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 254)       2341118   
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 16, 16, 254)       0         
                                                      

In [9]:
# Creating another network deeper, adding more Conv and Polling layers

deep_model = Sequential()

deep_model.add(Conv2D(1024, kernel_size=(3, 3), padding='same',input_shape=(new_im_size, new_im_size, channels)))
deep_model.add(LeakyReLU(alpha=0.1))
deep_model.add(MaxPooling2D((2, 2),padding='same'))
deep_model.add(Dropout(0.25))

deep_model.add(Conv2D(254, kernel_size=(3, 3), padding='same',input_shape=(new_im_size, new_im_size, channels)))
deep_model.add(LeakyReLU(alpha=0.1))
deep_model.add(MaxPooling2D((2, 2),padding='same'))
deep_model.add(Dropout(0.25))

deep_model.add(Conv2D(128, kernel_size=(3, 3), padding='same',input_shape=(new_im_size, new_im_size, channels)))
deep_model.add(LeakyReLU(alpha=0.1))
deep_model.add(MaxPooling2D((2, 2),padding='same'))
deep_model.add(Dropout(0.25))

deep_model.add(Conv2D(64, kernel_size=(3, 3), padding='same',input_shape=(new_im_size, new_im_size, channels)))
deep_model.add(LeakyReLU(alpha=0.1))
deep_model.add(MaxPooling2D((2, 2),padding='same'))
deep_model.add(Dropout(0.25))

deep_model.add(Flatten())
deep_model.add(Dense(64, activation='relu'))
deep_model.add(Dropout(0.25))
deep_model.add(Dense(num_classes, activation='softmax'))
               
deep_model.compile(loss=tensorflow.keras.losses.categorical_crossentropy, optimizer=tensorflow.keras.optimizers.Adam(),metrics=['accuracy'])

# Visualize the model through the summary function
deep_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 1024)      28672     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 32, 32, 1024)      0         
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 16, 16, 1024)      0         
 g2D)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 16, 16, 1024)      0         
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 254)       2341118   
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 16, 16, 254)       0         
                                                      

In [10]:
# Train the model

batch_size = 64
epochs = 5

scratch_model_history = scratch_model.fit(x_train, y_train_enc, batch_size=batch_size, shuffle=True, epochs=epochs, validation_data=(x_val, y_val_enc))

Epoch 1/5
 84/586 [===>..........................] - ETA: 13:18 - loss: 2.0514 - accuracy: 0.2679

In [ ]:
deep_model_history = deep_model.fit(x_train, y_train_enc, batch_size=batch_size, shuffle=True, epochs=epochs, validation_data=(x_val, y_val_enc))

In [ ]:
def plot_history(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

In [ ]:
# Getting the results

plot_history(scratch_model_history)

print("Training accuracy: ", accuracy_score(np.argmax(scratch_model.predict(x_train), axis=-1), y_train))
print("Validation accuracy: ", accuracy_score(np.argmax(scratch_model.predict(x_val), axis=-1), y_val))
print("Test accuracy: ", accuracy_score(np.argmax(scratch_model.predict(x_test), axis=-1), y_test))

In [ ]:
plot_history(deep_model_history)

print("Training accuracy: ", accuracy_score(np.argmax(deep_model.predict(x_train), axis=-1), y_train))
print("Validation accuracy: ", accuracy_score(np.argmax(deep_model.predict(x_val), axis=-1), y_val))
print("Test accuracy: ", accuracy_score(np.argmax(deep_model.predict(x_test), axis=-1), y_test))

In [ ]:
# Adding data augmentation

data_augmentation = tf.keras.Sequential([tensorflow.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
tensorflow.keras.layers.experimental.preprocessing.RandomRotation(0.2)])

model.add(data_augmentation)

### Pre-trained model ( Xception Net model )

In [ ]:
# Creating the model based over the pretrained Xception network
from tensorflow.keras import applications
import tensorflow
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
model = models.Sequential()

model.add(tensorflow.keras.layers.UpSampling2D(size=(7,7),input_shape=(32,32,3)))

Xception_model = applications.Xception(weights = "imagenet", include_top=False, input_shape = (224, 224, channels))

for layer in Xception_model.layers:
    layer.trainable = False

Inputs = layers.Input(shape=(32,32,3))
x = model(Inputs)
x = Xception_model(x)
x = layers.Flatten()(x)
# let's add a fully-connected layer
x = layers.Dense(128, activation='relu')(x)
# and a logistic layer for 10 classes
predictions = layers.Dense(10, activation='softmax')(x)

# this is the model we will train
pre_trained_model = tensorflow.keras.Model(Inputs, outputs=predictions)
pre_trained_model.compile(loss=tensorflow.keras.losses.categorical_crossentropy, optimizer=tensorflow.keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
# Visualize the model through the summary function
pre_trained_model.summary()

In [ ]:
# Let's train the model!
epochs = 5 #it may take a while, the number of epochs should be low...
pretrained_model_history = pre_trained_model.fit(x_train, y_train_enc, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val_enc))

In [ ]:
# Getting the results
plot_history(pretrained_model_history)

print("Training accuracy: ", accuracy_score(np.argmax(pre_trained_model.predict(x_train), axis=-1), y_train))
print("Validation accuracy: ", accuracy_score(np.argmax(pre_trained_model.predict(x_val), axis=-1), y_val))
print("Test accuracy: ", accuracy_score(np.argmax(pre_trained_model.predict(x_test), axis=-1), y_test))

#### Comparing the models

In [ ]:
# Create here the plots to compare the "from scratch" model and the "pretrained" model
# Try to produce a comparison plot about the accuracies (train and validation) and another plot for the losses
# Creating the plots to compare the "from scratch" model and the "pretrained" model
# Producing accuracy over epochs plot

scratch_model_train_acc = scratch_model_history.history['accuracy']
scratch_model_valid_acc = scratch_model_history.history['val_accuracy']
scratch_model_train_loss = scratch_model_history.history['loss']
scratch_model_valid_loss = scratch_model_history.history['val_loss']

pretrained_model_train_acc = pretrained_model_history.history['accuracy']
pretrained_model_valid_acc = pretrained_model_history.history['val_accuracy']
pretrained_model_train_loss = pretrained_model_history.history['loss']
pretrained_model_valid_loss = pretrained_model_history.history['val_loss']

print("Producing accuracy over epochs plot")
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(16,7))

plt.plot(scratch_model_train_acc, label="Scratch Train Acc.")
plt.plot(scratch_model_valid_acc, label="Scratch Valid. Acc.")

plt.plot(pretrained_model_train_acc, label="Pre-Trained Train Acc.")
plt.plot(pretrained_model_valid_acc, label="Pre-Trained Valid. Acc.")

plt.xlabel('Epoch')
plt.ylabel('Accuracy(%)')
plt.legend(loc='lower right', fancybox=True, shadow=True, ncol=4)
plt.grid()
plt.savefig('acc_epochs.png', dpi=300)


# Producing loss over epochs plot
print("Producing loss over epochs plot")
fig = plt.figure(figsize=(16,7))

plt.plot(scratch_model_train_loss, label="Scratch Train Loss")
plt.plot(scratch_model_valid_loss, label="Scratch Valid. Loss")

plt.plot(pretrained_model_train_loss, label="Pre-Trained Train Loss")
plt.plot(pretrained_model_valid_loss, label="Pre-Trained Valid. Loss")

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right', fancybox=True, shadow=True, ncol=4)
plt.grid()
plt.savefig('loss_epochs.png', dpi=300)

plt.show()